<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/%EA%B0%9C%EC%9D%B8%EC%97%B0%EA%B8%88(%EB%AA%A8%EB%A9%98%ED%85%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install finance-datareader

In [2]:
import pandas as pd
import FinanceDataReader as fdr
import re
import numpy as np
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
etfs = fdr.StockListing('ETF/KR')
rep_etfs = etfs[etfs['Name'].str.contains("TIGER", case=False)]


In [ ]:
import concurrent.futures

# ==============================
# 1. 기본 설정
# ==============================
# symbols = rep_etfs['Symbol'].tolist() # 대표 ETF 리스트
start_date = (datetime.datetime.today() - pd.DateOffset(months=9)).strftime('%Y-%m-%d')

# end_date를 오늘 날짜까지 설정하여 최대한 많은 데이터를 일단 가져옵니다.
# (10월 1일에 실행하면 10월 1일 데이터까지 포함)
end_date = datetime.datetime.today().strftime('%Y-%m-%d')

# ==============================
# 2. 데이터 조회 함수 (불완전한 현재 월 데이터 제거 로직 추가)
# ==============================
def fetch_monthly_data(sym, start, end):
    try:
        df = fdr.DataReader(sym, start=start, end=end)[['Open', 'Close']]

        # 💡 핵심 수정 로직: 불완전한 현재 월 데이터를 제거
        # 오늘 날짜 (마지막 인덱스)가 월초일 경우, 그 행을 제거하여 직전 월까지의 데이터만 남깁니다.
        # (10/1에 조회 시 10/1 데이터를 제거하여 9/30까지의 데이터만 남김)

        # 마지막 거래일의 월과 오늘 날짜의 월이 같다면, 마지막 행을 제거 (불완전한 현재 월 데이터)
        if df.index[-1].month == datetime.datetime.now().month and len(df) > 1:
             df = df.iloc[:-1]

        # 리샘플링 (마지막으로 제거되지 않은 날짜가 지난 달의 월말 종가임)
        df_monthly = pd.DataFrame({
            f'{sym}_Open': df['Open'].resample('MS').first(),
            f'{sym}_Close': df['Close'].resample('ME').last()
        })

        # 만약 df의 마지막 날짜가 9/30이고 today가 10/1이라면, 9월 월말 종가가 잘 계산됩니다.
        return df_monthly

    except Exception as e:
        print(f"{sym} 조회 실패: {e}")
        return None

# ==============================
# 3. 병렬 처리
# ==============================
all_data_list = []

max_workers = 8  # CPU 코어 수나 네트워크 상황에 맞게 조절
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(fetch_monthly_data, sym, start_date, end_date): sym for sym in symbols}
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result is not None:
            all_data_list.append(result)

In [ ]:
all_data_list

In [25]:
monthly_merged = pd.concat(all_data_list, axis=1)

# 월초 시가 / 월말 종가 분리
monthly_open = monthly_merged.filter(like='_Open').rename(columns=lambda x: x.replace('_Open',''))
monthly_close = monthly_merged.filter(like='_Close').rename(columns=lambda x: x.replace('_Close',''))

# 인덱스 변환 및 검증
monthly_open.index = pd.to_datetime(monthly_open.index, errors='coerce')
monthly_close.index = pd.to_datetime(monthly_close.index, errors='coerce')
assert monthly_open.index.equals(monthly_close.index), "Index mismatch between open and close"
# Combine monthly_open and monthly_close into monthly_price using combine
# We'll use the first non-NaN value at each position.
monthly_price = monthly_open.combine_first(monthly_close)

In [27]:
def get_top_n_momentum(monthly_price, etfs=None,
                       N_momentum=6, N_skip=2, top_n=3,
                       reference_date=None, use_filtered=True):
    """
    최근 기준 월말(reference_date) 기준 top_n 모멘텀 종목 선택
    """
    # 1️⃣ 필터링
    if use_filtered and etfs is not None:
        exclude_pattern = "레버리지|인버스|2X|커버드콜|혼합"
        exclude_symbols = etfs[etfs['Name'].str.contains(exclude_pattern, regex=True)]['Symbol'].tolist()
        monthly_price = monthly_price.drop(columns=[c for c in monthly_price.columns if c in exclude_symbols])

    monthly_price = monthly_price.sort_index()
    month_end = monthly_price.resample('M').last()

    # 2️⃣ 기준 월말
    ref = pd.to_datetime(reference_date) + pd.offsets.MonthEnd(0)
    if ref not in month_end.index:
        raise ValueError(f"{reference_date} 는 월말 데이터에 없음.")

    i = month_end.index.get_loc(ref)
    if i < (N_momentum + N_skip):
        raise ValueError("데이터가 부족합니다. 더 긴 기간 필요.")

    # 3️⃣ 모멘텀 계산용 기간
    start_idx = i - (N_momentum + N_skip)
    end_idx = i - N_skip
    momentum_window = month_end.iloc[start_idx:end_idx+1]  # 끝 포함

    # 4️⃣ 모멘텀 점수: 마지막 달 / 첫 달 - 1
    momentum_scores = (momentum_window.iloc[-1] / momentum_window.iloc[0]) - 1

    # 5️⃣ 상위 top_n 선택
    top_symbols = momentum_scores.nlargest(top_n).index
    top_scores = momentum_scores[top_symbols]

    # 이름 매핑
    if etfs is not None:
        names = etfs.set_index("Symbol").loc[top_symbols, "Name"].tolist()
    else:
        names = list(top_symbols)

    # 6️⃣ DataFrame 반환
    df_top = pd.DataFrame({
        "ReferenceMonth": [ref.strftime("%Y-%m")],
        "NextMonth": [(ref + pd.offsets.MonthEnd(1)).strftime("%Y-%m")],
        "Selected": [list(top_symbols)],
        "Names": [names],
        "Momentum": [list(top_scores.round(4))]
    })

    return df_top

In [29]:
top3_df = get_top_n_momentum(monthly_price, etfs,
                             N_momentum=6, N_skip=2, top_n=3,
                             reference_date="2025-09")
print(top3_df)

  ReferenceMonth NextMonth                  Selected  \
0        2025-09   2025-10  [463250, 139230, 494670]   

                                          Names                 Momentum  
0  [TIGER K방산&우주, TIGER 200 중공업, TIGER 조선TOP10]  [1.5865, 0.863, 0.6575]  
